In [1]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_APP_GOOGLE = db['TRAVEL_APPLICATION']
collection_APP_APPLE = db['TRAVEL_APPLICATION_APPLE']

data_APP_GOOGLE = collection_APP_GOOGLE.find()
data_APP_APPLE = collection_APP_APPLE.find()

df_APP_GOOGLE = pd.DataFrame(data_APP_GOOGLE)
df_APP_APPLE = pd.DataFrame(data_APP_APPLE)

In [3]:
# app(google, apple) 데이터를 concat 진행
df_app = pd.concat([df_APP_GOOGLE, df_APP_APPLE], ignore_index=True)

In [4]:
# 데이터 사이즈 확인
df_app.shape

(17751, 6)

In [5]:
# 데이터 확인
df_app.head()

,_id,app_name,user_score,user_date,user_comments,evaluation
0,662f099973f7ce16925e6b4e,tripadvisor,4,2019년 5월 13일,낯선 여행지에서 맞든 틀리든 이런 가이드가 있다는 것은 많은 도움이 됩니다. 때론 ...,73
1,662f099973f7ce16925e6b4f,tripadvisor,2,2019년 12월 16일,웹사이트에서 구글로 로긴하고 예약한다음 메일에 티켓 다운로드하기가 있길래 폰에서 앱...,59
2,662f099a73f7ce16925e6b50,tripadvisor,1,2020년 3월 20일,최근 업데이트후 실시간 항공권 가격이 낚시성 가격으로 나오고 결제는 실제로 되지 않...,60
3,662f099a73f7ce16925e6b51,tripadvisor,4,2019년 5월 13일,여행전 항상 여행정보를 위해 열어보는 앱입니다. 숙소및 식당은 물론이거니와 들러볼만...,8
4,662f099a73f7ce16925e6b52,tripadvisor,5,2019년 5월 19일,"다양한 콘텐츠와 뷰, 리뷰가 잘 어울어진 어플. 하지만, 지극히 여행자 중심의 결과...",14


In [7]:
# 한글 자연어 처리기 설치
!pip install konlpy    
!pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.5 MB/s eta 0:00:0000:01


In [8]:
list_review = df_app["user_comments"].to_list()
list_review

['낯선 여행지에서 맞든 틀리든 이런 가이드가 있다는 것은 많은 도움이 됩니다. 때론 기대하지 않았던 정보도 얻을 수 있습니다. 정보의 정확성을 기하기 위해 여기에서 추천한 공을 다른 방법으로 검색해보면 어느 정도 유추가 가능해집니다. 여행 때마다 요긴하게 활용합니다.',
 '웹사이트에서 구글로 로긴하고 예약한다음 메일에 티켓 다운로드하기가 있길래 폰에서 앱을 다운로드하고 같은 구글 계정으로 로그인 햇는데 예약 사항을 전혀 확인할수 없고 티켓조차 앱으로 열리지 않음 메일에서 예약 상세보기 하려다가 잘못 눌러서 취소하기 눌러서 바로 예약이 취소되버렸음 취소할거냐고 확인조차 하지 않고 바로 취소되버려서 너무 황당함... 시즌이라 액티비티 예약마감 다찼고 내가 예약했던 날짜도 다시 예약이 안되는 상황임. 문의메일보내고 기다리는 중인데 너무 짜증남 트립어드바이저 자체 서비스는 좋은데 이 앱과 웹 연동이 잘 안되는듯 앱에서 내가 예약한 엑티비티는 추천 목록에서 볼려고 누르기만 해도 자꾸 오류난다고 로드도 안됨',
 '최근 업데이트후 실시간 항공권 가격이 낚시성 가격으로 나오고 결제는 실제로 되지 않는 경우가 많아져 실망했음.. 최저가 업체 클릭시 업체가 제공된 항공권 가격은 사라지고 다시 검색하라며 클릭 유도함... 거지 같이 업데이트 되어서 정말 실망함. 그동안 몇 번 잘 사용 한건 고맙게 생각합니다. 쿠키와 개인정보 활용으로 갑자기 해외 업체에서 전화와서 비행기표 직접 예약 해주겠다는 새끼도 있었습니다. 조심하세요.',
 '여행전 항상 여행정보를 위해 열어보는 앱입니다. 숙소및 식당은 물론이거니와 들러볼만한 랜드마크등에 대한 정보를 확인하고 계획을 잡는데 큰 도움이 되고 있어요. but, 단점은 별로 정보가 없어서 종종 아쉬움이 남기도 합니다.',
 '다양한 콘텐츠와 뷰, 리뷰가 잘 어울어진 어플. 하지만, 지극히 여행자 중심의 결과라 로컬의 맛집과 시설을 별도로 필터링 할수 있는 기능이 추가되면 좋을듯 함.',
 '여행갈때마다 이용하는데 계획 세울때 유용하게 썼습

In [ ]:
from textblob import TextBlob

# 분석할 문장들의 리스트 = list_review

# 감정 분석 결과를 저장할 딕셔너리 초기화
sentiment_review = {'Positive': 0, 'Negative': 0, 'Neutral': 0}

# 각 문장에 대한 감정 분석
for sentence in list_review:
    sentence_str = str(sentence)
    blob = TextBlob(sentence_str)
    sentiment = blob.sentiment.polarity
    # 감정 평가를 바탕으로 빈도수 계산
    if sentiment > 0:
        sentiment_review['Positive'] += 1
    elif sentiment < 0:
        sentiment_review['Negative'] += 1
    else:
        sentiment_review['Neutral'] += 1

# 결과 출력
print(f"긍정적 문장의 수: {sentiment_review['Positive']}")
print(f"부정적 문장의 수: {sentiment_review['Negative']}")
print(f"중립적 문장의 수: {sentiment_review['Neutral']}")